In [1]:
# batcher's even odd sort

class BatcherOddEvenMerge:
    
    def __init__(sefl):
        pass
    
    def sort_aux(self,a,b):
        
        if len(a)==1:
            return [min(a[0],b[0]),max(a[0],b[0])]

        # using batcher's numbering for odd and even 
        # (contrary to based at zero arrays)
        a_odd = a[0::2]
        a_even = a[1::2]
        b_odd = b[0::2]
        b_even = b[1::2]

        odd = self.sort_aux(a_odd,b_odd)
        even = self.sort_aux(a_even,b_even)

        c = [0]*(len(a)+len(b))
        c[0] = odd[0]
        c[-1] = even[-1]
        c[1:-1:2] = odd[1::]
        c[2:-1:2] = even[0:len(even)-1]
        for i in range(1,len(c)-1,2):
            if c[i]>c[i+1] : c[i],c[i+1]=c[i+1],c[i]
        return c
    
    def sort(self,a,b):
        
        def power_of_two(n):
            while n>1:
                if n%2==1:
                    return False
                n //= 2
            return True
        
        assert len(a)==len(b), 'lists must be of equal size'
        assert power_of_two(len(a)), 'list length must be a power of 2'
        return self.sort_aux(a,b)

In [2]:
import random

k= 16
a = [i for i in range(k)]
#random.shuffle(a)
print(a)
b = [i for i in range(k)]
#random.shuffle(b)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [3]:
bod = BatcherOddEvenMerge()
bod.sort(a,b)

[0,
 0,
 1,
 1,
 2,
 2,
 3,
 3,
 4,
 4,
 5,
 5,
 6,
 6,
 7,
 7,
 8,
 8,
 9,
 9,
 10,
 10,
 11,
 11,
 12,
 12,
 13,
 13,
 14,
 14,
 15,
 15]

In [4]:
a = [0,0,0,0,1,1,1,1]
b = [0,1,1,1,1,1,1,1]
bod.sort(a,b)


[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]